# Семинар 7. Линейные методы. Vowpal Wabbit.

by Ilya Irkhin

Vowpal Wabbit on GitHub: https://github.com/JohnLangford/vowpal_wabbit

Vowpal Wabbit Tutorial: https://github.com/JohnLangford/vowpal_wabbit/wiki/Tutorial

In [1]:
import pandas as pd
import numpy as np

In [4]:
# https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow/data
train_path = './data/train-sample.csv'

In [5]:
data = pd.read_csv(train_path)
data.head()

,PostId,PostCreationDate,OwnerUserId,OwnerCreationDate,ReputationAtPostCreation,OwnerUndeletedAnswerCountAtPostTime,Title,BodyMarkdown,Tag1,Tag2,Tag3,Tag4,Tag5,PostClosedDate,OpenStatus
0,10035653,04/05/2012 20:37:35,1159226,01/19/2012 18:46:16,1,0,what is the best way to connect my application...,I know this question can be answered by search...,c++,NaN,NaN,NaN,NaN,04/05/2012 23:31:34,too localized
1,8922537,01/19/2012 07:38:27,1157921,01/19/2012 07:31:34,1,0,upload a zip file and get xml file information...,i make a up-loader to `upload a zip file with ...,php,xml,cakephp,zip,NaN,01/19/2012 16:43:00,not a real question
2,5962216,05/11/2011 09:43:05,696219,04/07/2011 06:38:45,40,2,UIButton Background Change on click,Can anyone tell me how to change UIButton back...,iphone-sdk-4.0,NaN,NaN,NaN,NaN,NaN,open
3,10070625,04/09/2012 08:16:57,490895,10/29/2010 04:59:18,1,1,using the pt_regs struct,I've recently started getting into linux kerne...,linux,module,kernel,NaN,NaN,NaN,open
4,8960935,01/22/2012 12:10:17,1017103,10/27/2011 18:26:24,28,0,Can I use LIMIT function in SQL line where it'...,Can I use LIMIT function in SQL line where it'...,mysql,limit,NaN,NaN,NaN,NaN,open


In [9]:
print( data.OpenStatus[10])
print( '\n\n\n')
print(data.BodyMarkdown[10])

open




I have my own jquery plugin for my app. Its working fine for first callbacks and the second callbacks are not assigning.

    $('element1').myplugin({callback:foo});
    $('element2').myplugin({callback:bla});

second callback not working always calling 'foo'.

plugin code:

    (function($) {
    	$.fn.myplugin = function(options) {
              options.callback();
            }
    });


In [10]:
data_train = data.iloc[:50000, :]
data_test = data.iloc[70000:, :]

In [11]:
# трансформируем данные для VW
import re

def save_to_vw(data, fname):
    with open(fname, 'w') as fout:
        for _, row in data.iterrows():
            text = filter(lambda x: len(x) > 1, re.split("[^a-z]",
                                    row.BodyMarkdown.lower()))
            text = ' '.join(text)
            if row.OpenStatus == 1:
                target = 1
            else:
                target = -1
            fout.write('{0} |n 0:{1} {2} |t {3}\n'.format(target, 
                                        row.ReputationAtPostCreation,
                                        row.Tag1,
                                        text))

In [13]:
save_to_vw(data_train, './data/train.vw')
save_to_vw(data_test, './data/test.vw')

In [14]:
!head -n 2 train.vw

'head' is not recognized as an internal or external command,
operable program or batch file.


In [15]:
!vw -d train.vw -c -k -f model.vw --passes 10 

'vw' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
!vw -d test.vw -i model.vw -t -p pred.txt

only testing
predictions = pred.txt
Num weight bits = 18
learning rate = 10
initial_t = 1
power_t = 0.5
using no cache
Reading datafile = test.vw
num sources = 1
average    since         example     example  current  current  current
loss       last          counter      weight    label  predict features
1.152298   1.152298          1      1.0    -1.0000   0.0735       63
1.548524   1.944750          2      2.0    -1.0000   0.3945       31
1.304907   1.061290          4      4.0     1.0000  -0.3451      103
0.919371   0.533836          8      8.0     1.0000   0.7522       33
0.708812   0.498253         16     16.0    -1.0000  -1.0000      139
0.694174   0.679535         32     32.0     1.0000   0.0276      154
0.757704   0.821234         64     64.0     1.0000  -0.3299       36
0.843794   0.929883        128    128.0     1.0000   1.0000      108
0.821948   0.800103        256    256.0     1.0000   0.4271       30
0.818219   0.814491        512    512.0     1.0000   0.5219       16
0.80

In [11]:
!head -n 10 pred.txt

0.073451
0.394543
-0.440162
-0.345051
1.000000
0.707419
-0.410080
0.752177
0.555422
0.082952


In [12]:
from sklearn.metrics import roc_auc_score

def calc_vw_qual():
    preds = pd.read_csv('pred.txt', header=None).iloc[:, 0].values
    target = data_test.OpenStatus.values
    print (roc_auc_score(target, preds))

In [13]:
calc_vw_qual()

0.773175444484


In [18]:
!vw -d train.vw -c -k -f model.vw --passes 10 -l 0.1
!vw -d test.vw -i model.vw -t -p pred.txt
print ('\n\n\n')
calc_vw_qual()

final_regressor = model.vw
Num weight bits = 18
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average    since         example     example  current  current  current
loss       last          counter      weight    label  predict features
1.000000   1.000000          1      1.0    -1.0000   0.0000      254
0.824177   0.648353          2      2.0    -1.0000  -0.1948       44
1.239373   1.654570          4      4.0    -1.0000  -0.1296       82
1.074697   0.910021          8      8.0     1.0000  -0.2473       77
0.666821   0.258945         16     16.0    -1.0000  -0.5618      190
0.758021   0.849221         32     32.0     1.0000  -0.0742       21
0.889370   1.020718         64     64.0     1.0000   0.2461      100
0.948542   1.007714        128    128.0     1.0000  -0.0434       52
0.957862   0.967181        256    256.0    -1.0000   0.1023      279
0.921585   0.885309        512    

In [20]:
!vw -d train.vw -c -k -f model.vw --passes 10 -l 0.1 --ngram t2
!vw -d test.vw -i model.vw -t -p pred.txt
print ('\n\n\n')
calc_vw_qual()

Generating 2-grams for t namespaces.
final_regressor = model.vw
Num weight bits = 18
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average    since         example     example  current  current  current
loss       last          counter      weight    label  predict features
1.000000   1.000000          1      1.0    -1.0000   0.0000      504
0.893775   0.787549          2      2.0    -1.0000  -0.1126       84
1.116160   1.338546          4      4.0    -1.0000  -0.0854      160
1.027996   0.939831          8      8.0     1.0000  -0.1864      150
0.708380   0.388764         16     16.0    -1.0000  -0.4080      376
0.781467   0.854553         32     32.0     1.0000  -0.0693       38
0.906742   1.032018         64     64.0     1.0000   0.1270      196
0.954535   1.002328        128    128.0     1.0000   0.0251      100
0.958956   0.963377        256    256.0    -1.0000  -0.0042      5

In [21]:
!vw -d train.vw -c -k -f model.vw --passes 10 -l 0.1 --ngram t2 --skips t2
!vw -d test.vw -i model.vw -t -p pred.txt
print ('\n\n\n')
calc_vw_qual()

Generating 2-grams for t namespaces.
Generating 2-skips for t namespaces.
final_regressor = model.vw
Num weight bits = 18
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average    since         example     example  current  current  current
loss       last          counter      weight    label  predict features
1.000000   1.000000          1      1.0    -1.0000   0.0000     1001
0.932438   0.864876          2      2.0    -1.0000  -0.0700      161
1.069380   1.206322          4      4.0    -1.0000  -0.0552      313
1.013547   0.957714          8      8.0     1.0000  -0.1455      293
0.764101   0.514655         16     16.0    -1.0000  -0.2995      745
0.818270   0.872439         32     32.0     1.0000  -0.0604       69
0.933228   1.048186         64     64.0     1.0000   0.0474      385
0.968953   1.004679        128    128.0     1.0000   0.0376      193
0.972765   0.976577        25

In [22]:
!vw -d train.vw -c -k -f model.vw --passes 10 -l 0.1 --ngram t2 -b 28
!vw -d test.vw -i model.vw -t -p pred.txt
print( '\n\n\n')
calc_vw_qual()

Generating 2-grams for t namespaces.
final_regressor = model.vw
Num weight bits = 28
learning rate = 0.1
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average    since         example     example  current  current  current
loss       last          counter      weight    label  predict features
1.000000   1.000000          1      1.0    -1.0000   0.0000      504
0.893775   0.787549          2      2.0    -1.0000  -0.1126       84
1.116160   1.338546          4      4.0    -1.0000  -0.0854      160
1.027811   0.939461          8      8.0     1.0000  -0.1863      150
0.709204   0.390598         16     16.0    -1.0000  -0.4044      376
0.781565   0.853925         32     32.0     1.0000  -0.0694       38
0.907234   1.032904         64     64.0     1.0000   0.1415      196
0.956017   1.004799        128    128.0     1.0000   0.0238      100
0.958802   0.961587        256    256.0    -1.0000   0.0139      5